In [1]:
import pandas as pd
import numpy as np

# 1. to define the datasets
to divide it into train and test sets.

In [13]:
# to define the datasets

dataset = pd.read_csv('watermelon3.0a.csv', sep=',')

dataset = np.array(dataset)

# print('before')
# print(dataset)

# translate '是' and '否' into 1 and 0
for i in range(len(dataset)):
    dataset[i][3] = 1 if dataset[i][3] == '是' else 0


dataset = dataset.T
dataset = dataset[1:4]
# print('after altering')
print(dataset)

print(dataset.shape)

d = int(17 * 0.7)

# [0:2, 0:d] [2:3, 0:d] [0:2, d:] [2:3, d:]
train_x = dataset[0:2, 0:d]
train_y = dataset[2:3, 0:d]
test_x = dataset[0:2, d:]
test_y = dataset[2:3, d:]

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)


[[0.697 0.774 0.634 0.608 0.556 0.403 0.481 0.437 0.666 0.243 0.245 0.343
  0.639 0.657 0.36 0.593 0.719]
 [0.46 0.376 0.264 0.318 0.215 0.237 0.149 0.211 0.091 0.267 0.057 0.099
  0.161 0.198 0.37 0.042 0.103]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]]
(3, 17)
(2, 11) (1, 11) (2, 6) (1, 6)


# 2. initialize the parameters

In [14]:
# initialize the parameters

def initialize_with_zeros(dim : int):
    return np.random.randn(dim, 1) * 0.001

initialize_with_zeros(4)

array([[ 0.0020741 ],
       [-0.0014076 ],
       [-0.00031198],
       [-0.00030822]])

# 3. propagate

In [22]:
def propagate(Beta:np.ndarray, X:np.ndarray, Y:np.ndarray):
    L = np.sum(-1 * Y * np.dot(Beta.T, X) + np.log(1 + np.exp(np.dot(Beta.T, X))))
    p1 = np.exp(np.dot(Beta.T, X)) / (1 + np.exp(np.dot(Beta.T, X)))
    db = -1 * np.sum(X * (Y - p1))
    dbb = np.sum(np.dot(X.T, X) * p1 * (1 - p1))

    grads = {
        'db': db,
        'dbb': dbb
    }

    return grads


# 4. Optimize